In [1]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import TFSMLayer
from tensorflow.keras.models import load_model
from tensorflow.keras.saving import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import tensorflow

In [2]:
from tensorflow.keras.layers import TFSMLayer
model = TFSMLayer(r"C:\Users\HP\mask_detector_model", call_endpoint='serve')

In [3]:
model

<TFSMLayer name=tfsm_layer, built=True>

In [ ]:
import cv2
import numpy as np
import imutils
from imutils.video import VideoStream
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import TFSMLayer

# Load the SavedModel using TFSMLayer
saved_model_path = r"C:\Users\HP\mask_detector_model"
model = TFSMLayer(saved_model_path, call_endpoint='serve')

# Define constants
CONFIDENCE_THRESHOLD = 0.5
IMG_SIZE = 224

# Initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

# Function to detect faces and mask status
def detect_and_predict_mask(frame):
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 177.0, 123.0))

    faces = []
    locs = []
    preds = []

    # Assuming the model is detecting faces directly
    detections = model(blob)  # Run the model inference
    
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > CONFIDENCE_THRESHOLD:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            startX, startY = max(0, startX), max(0, startY)
            endX, endY = min(w - 1, endX), min(h - 1, endY)

            face = frame[startY:endY, startX:endX]
            if face.size == 0:
                continue
            
            # Resize the face to the required size
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)  # Ensure the shape is (1, 224, 224, 3)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        # Stack faces and pass them to the model
        faces = np.vstack(faces)
        preds = model(faces)

    return locs, preds
# Main loop for video processing
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    # Preprocess the frame to match the model's expected input
    face = cv2.resize(frame, (224, 224))
    face = img_to_array(face)
    face = preprocess_input(face)
    face = np.expand_dims(face, axis=0)  # Shape becomes (1, 224, 224, 3)
    
    # Predict using the model
    preds = model(face)
    
    # Extract the probabilities for each class
    mask_prob, without_mask_prob = preds[0]
    
    # Determine the class label and color for the bounding box
    label = "Mask" if mask_prob > without_mask_prob else "No Mask"
    color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
    
    # Display the label on the frame
    cv2.putText(frame, f"{label}: {max(mask_prob, without_mask_prob) * 100:.2f}%", 
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.85, color, 2)
    
    # Show the output frame
    cv2.imshow("Frame", frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cv2.destroyAllWindows()
vs.stop()


[INFO] starting video stream...
